In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import calendar
import os
import re

In [5]:
def process_ppi_files(filepaths):
    """
    Reads multiple text files that each contain columns like:
        series id | year | period | value | footnotes
    and combines them into a single DataFrame covering 2003–2025.
    
    Steps:
      1) Remove table border lines (lines starting with '+').
      2) Parse the first non-removed line as the header.
      3) Convert 'year' and 'value' to numeric; extract 'month' from 'period'.
      4) Rename 'value' to 'ppi'.
      5) Sort by (year, month) across *all* combined data.
      6) Compute month-over-month proportional change (ppi_pro_change) across the entire timespan.
      7) Expand monthly rows into daily rows.
      8) Extract the NACIS code from the first file's name (or verify all have same NACIS).
      9) Save the final daily DataFrame to 'data/processed/PPI{nacis}(03-25).csv'.
      
    Returns:
      The combined daily DataFrame (covering 03-25).
    """
    
    
    all_dfs = []
    
    for filepath in filepaths:
        # Read and parse each file individually
        
        with open(filepath, 'r') as f:
            lines = f.readlines()
        
        # Remove lines that start with '+'
        data_lines = [line for line in lines if not line.startswith('+')]
        
        data_clean = []
        for line in data_lines:
            line_strip = line.strip()
            if not line_strip:
                continue
            parts = [x.strip() for x in line_strip.strip('|').split('|')]
            data_clean.append(parts)
        
        header = data_clean[0]
        rows = data_clean[1:]
        df_temp = pd.DataFrame(rows, columns=header)
        
        # Check columns
        needed_cols = {'year', 'period', 'value'}
        if not needed_cols.issubset(df_temp.columns):
            print(f"ERROR: Missing columns in {filepath}. Found: {df_temp.columns.tolist()}")
            continue
        
        # Convert columns
        df_temp['year'] = df_temp['year'].astype(int)
        df_temp['value'] = df_temp['value'].astype(float)
        
        # Extract month
        df_temp['month'] = df_temp['period'].str.extract(r'M(\d+)').astype(int)
        
        # Rename 'value' -> 'ppi'
        df_temp.rename(columns={'value': 'ppi'}, inplace=True)
        
        # Keep only relevant columns
        df_temp = df_temp[['year', 'month', 'ppi']]
        
        all_dfs.append(df_temp)
    
    
    combined_df = pd.concat(all_dfs, ignore_index=True)
    combined_df.sort_values(by=['year', 'month'], inplace=True)
    

    combined_df['ppi_pro_change'] = combined_df['ppi'].pct_change()
    
    # Now we expand monthly data to daily
    daily_rows = []
    for _, row in combined_df.iterrows():
        year = int(row['year'])
        month = int(row['month'])
        days_in_month = calendar.monthrange(year, month)[1]
        start_date = datetime(year, month, 1)
        
        date_range = pd.date_range(start=start_date, periods=days_in_month)
        for single_day in date_range:
            daily_rows.append({
                'date': single_day,
                'ppi': row['ppi'],
                'ppi_pro_change': row['ppi_pro_change']
            })
    
    daily_df = pd.DataFrame(daily_rows)
    
    # Extract NACIS code from the *first* file in filepaths
    basename = os.path.basename(filepaths[0])
    match = re.search(r'PCU(\d+)', basename)
    nacis = match.group(1) if match else "unknown"
        
    out_dir = "../data/processed"
    os.makedirs(out_dir, exist_ok=True)
    out_filename = f"PPI{nacis}(03-25).csv"
    out_path = os.path.join(out_dir, out_filename)
    
    daily_df.to_csv(out_path, index=False)

    
    return daily_df

In [6]:
if __name__ == "__main__":
    files_in_chron_order = [
        "../data/raw/ppi_and_cpi/PCU314---314---(03-12).txt",
        "../data/raw/ppi_and_cpi/PCU314---314---(12-20).txt",
        "../data/raw/ppi_and_cpi/PCU314---314---(20-25).txt"
    ]
    # Process and combine them
    result_df = process_ppi_files(files_in_chron_order)
    if result_df is not None:
        print("\n=== Preview of combined daily DataFrame (first 10 rows) ===")
        print(result_df.head(10))



=== Preview of combined daily DataFrame (first 10 rows) ===
        date    ppi  ppi_pro_change
0 2003-12-01  100.0             NaN
1 2003-12-02  100.0             NaN
2 2003-12-03  100.0             NaN
3 2003-12-04  100.0             NaN
4 2003-12-05  100.0             NaN
5 2003-12-06  100.0             NaN
6 2003-12-07  100.0             NaN
7 2003-12-08  100.0             NaN
8 2003-12-09  100.0             NaN
9 2003-12-10  100.0             NaN
